# Process BLASTP queries 

Compare all results from one Gene to all other Isoforms

In [5]:
import sqlite3
import re
from typing import List, Dict, Tuple
import mysql.connector


In [2]:


def parse_blastp_file(file_path: str) -> Tuple[List[Dict], List[Dict], List[Dict], List[Dict]]:
    """
    Parse the BLASTP results file and extract data for all tables.
    
    Returns:
        Tuple containing lists of dictionaries for:
        (queries_data, genes_data, isoforms_data, results_data)
    """
    queries_data = []
    genes_data = []
    isoforms_data = []
    results_data = []
    
    current_query = None
    query_counter = 1
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
                
            # Parse query header
            if line.startswith('# Query:'):
                # Extract query information
                query_text = line[9:].strip()
                
                # Use regex to extract components from the query text
                pattern = r'([^\s]+)\s+pep\s+scaffold:([^:]+:[^:]+:[^:]+:[^:]+:[^:]+)\s+gene:([^\s]+)\s+transcript:([^\s]+)\s+gene_biotype:[^\s]+\s+transcript_biotype:[^\s]+\s+gene_symbol:([^\s]+)'
                match = re.search(pattern, query_text)
                
                if match:
                    query_id = match.group(1)
                    chrom_pos = match.group(2)
                    gene_id = match.group(3)
                    transcript_id = match.group(4)
                    gene_symbol = match.group(5)
                    
                    # Add to queries table
                    queries_data.append({
                        'id': query_counter,
                        'query': query_text,
                        'gene_id': gene_id,
                        'transcript_id': transcript_id
                    })
                    
                    # Add to genes table (if not already present)
                    gene_exists = any(g['Gene'] == gene_id for g in genes_data)
                    if not gene_exists:
                        genes_data.append({
                            'Gene': gene_id,
                            'chrom_pos': chrom_pos
                        })
                    
                    # Add to isoforms table
                    isoforms_data.append({
                        'transcripts': transcript_id,
                        'gene_id': gene_id
                    })
                    
                    current_query = query_counter
                    query_counter += 1
            
            # Parse results lines (not comments and not empty)
            elif not line.startswith('#') and current_query is not None:
                parts = line.split(',')
                if len(parts) >= 6:
                    # Split PDB ID and chain (format like 8SWH_L)
                    pdb_full = parts[1]
                    pdb_id = pdb_full[:4]  # First 4 chars are PDB ID
                    chain = pdb_full[5:] if len(pdb_full) > 4 else ''  # After underscore is chain
                    
                    results_data.append({
                        'query_id': current_query,
                        'pdb_id': pdb_id,
                        'Chain': chain,
                        'e_val': parts[2],
                        'query_cov': None if parts[4] == 'N/A' else float(parts[4])
                    })
    
    return queries_data, genes_data, isoforms_data, results_data

def create_database(db_path: str, 
                   queries_data: List[Dict], 
                   genes_data: List[Dict], 
                   isoforms_data: List[Dict], 
                   results_data: List[Dict]) -> None:
    """
    Create SQLite database and populate with the parsed data.
    """
    # Connect to SQLite database (will be created if it doesn't exist)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS queries (
        id INTEGER PRIMARY KEY,
        query TEXT,
        gene_id VARCHAR,
        transcript_id VARCHAR UNIQUE
    )
    """)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS genes (
        Gene VARCHAR PRIMARY KEY,
        chrom_pos VARCHAR
    )
    """)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS isoforms (
        transcripts VARCHAR PRIMARY KEY,
        gene_id VARCHAR
    )
    """)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        query_id INTEGER,
        pdb_id VARCHAR,
        Chain VARCHAR,
        e_val VARCHAR,
        query_cov REAL,
        FOREIGN KEY (query_id) REFERENCES queries(id)
    )
    """)
    
    # Create indexes for performance
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_results_query_id ON results(query_id)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_isoforms_gene_id ON isoforms(gene_id)")
    
    # Insert data
    # Insert genes first (referenced by other tables)
    for gene in genes_data:
        cursor.execute("""
        INSERT OR IGNORE INTO genes (Gene, chrom_pos)
        VALUES (?, ?)
        """, (gene['Gene'], gene['chrom_pos']))
    
    # Insert queries
    for query in queries_data:
        cursor.execute("""
        INSERT INTO queries (id, query, gene_id, transcript_id)
        VALUES (?, ?, ?, ?)
        """, (query['id'], query['query'], query['gene_id'], query['transcript_id']))
    
    # Insert isoforms
    for isoform in isoforms_data:
        cursor.execute("""
        INSERT OR IGNORE INTO isoforms (transcripts, gene_id)
        VALUES (?, ?)
        """, (isoform['transcripts'], isoform['gene_id']))
    
    # Insert results
    for result in results_data:
        cursor.execute("""
        INSERT INTO results (query_id, pdb_id, Chain, e_val, query_cov)
        VALUES (?, ?, ?, ?, ?)
        """, (result['query_id'], result['pdb_id'], result['Chain'], result['e_val'], result['query_cov']))
    
    # Commit changes and close connection
    conn.commit()
    conn.close()

def main():
    # Input and output paths
    input_file = 'test_job.out'  # Change to your input file path
    output_db = 'blastp_results.db'  # Change to your desired output database path
    
    # Parse the input file
    queries, genes, isoforms, results = parse_blastp_file(input_file)
    
    # Create the database
    create_database(output_db, queries, genes, isoforms, results)
    
    print(f"Database created successfully at {output_db}")
    print(f"Loaded {len(queries)} queries, {len(genes)} genes, {len(isoforms)} isoforms, and {len(results)} results.")

if __name__ == '__main__':
    main()

Database created successfully at blastp_results.db
Loaded 771 queries, 305 genes, 771 isoforms, and 172793 results.


In [12]:
import re
import mysql.connector
from mysql.connector import Error
from typing import List, Dict, Tuple

def parse_blastp_file(file_path: str) -> Tuple[List[Dict], List[Dict], List[Dict], List[Dict]]:
    """
    Parse the BLASTP results file and extract data for all tables.
    
    Returns:
        Tuple containing lists of dictionaries for:
        (queries_data, genes_data, isoforms_data, results_data)
    """
    queries_data = []
    genes_data = []
    isoforms_data = []
    results_data = []
    
    current_query = None
    query_counter = 1
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
                
            # Parse query header
            if line.startswith('# Query:'):
                # Extract query information
                query_text = line[9:].strip()
                
                # Use regex to extract components from the query text
                pattern = r'([^\s]+)\s+pep\s+scaffold:([^:]+:[^:]+:[^:]+:[^:]+:[^:]+)\s+gene:([^\s]+)\s+transcript:([^\s]+)\s+gene_biotype:[^\s]+\s+transcript_biotype:[^\s]+\s+gene_symbol:([^\s]+)'
                match = re.search(pattern, query_text)
                
                if match:
                    query_id = match.group(1)
                    chrom_pos = match.group(2)
                    gene_id = match.group(3)
                    transcript_id = match.group(4)
                    gene_symbol = match.group(5)
                    
                    # Add to queries table
                    queries_data.append({
                        'id': query_counter,
                        'query': query_text,
                        'gene_id': gene_id,
                        'transcript_id': transcript_id
                    })
                    
                    # Add to genes table (if not already present)
                    gene_exists = any(g['Gene'] == gene_id for g in genes_data)
                    if not gene_exists:
                        genes_data.append({
                            'Gene': gene_id,
                            'chrom_pos': chrom_pos
                        })
                    
                    # Add to isoforms table
                    isoforms_data.append({
                        'transcripts': transcript_id,
                        'gene_id': gene_id
                    })
                    
                    current_query = query_counter
                    query_counter += 1
            
            # Parse results lines (not comments and not empty)
            elif not line.startswith('#') and current_query is not None:
                parts = line.split(',')
                if len(parts) >= 6:
                    #print(parts)
                    # Split PDB ID and chain (format like 8SWH_L)
                    pdb_full = parts[1]
                    pdb_id = pdb_full[:4]  # First 4 chars are PDB ID
                    chain = pdb_full[5:] if len(pdb_full) > 4 else ''  # After underscore is chain
                    
                    results_data.append({
                        'query_id': current_query,
                        'pdb_id': pdb_id,
                        'Chain': chain,
                        'e_val': parts[2],
                        'query_cov': None if parts[5] == 'N/A' else float(parts[5])
                    })
    
    return queries_data, genes_data, isoforms_data, results_data

def create_mysql_connection(host_name: str, user_name: str, user_password: str, db_name: str):
    """Create a connection to MySQL database"""
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    
    return connection

def create_database(connection, db_name: str):
    """Create a new database"""
    cursor = connection.cursor()
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        print(f"Database '{db_name}' created successfully")
    except Error as err:
        print(f"Error: '{err}'")

def create_tables(connection):
    """Create all tables in the database"""
    cursor = connection.cursor()
    
    try:
        # Create genes table first (referenced by other tables)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS genes (
            Gene VARCHAR(255) PRIMARY KEY,
            chrom_pos VARCHAR(255)
        )
        """)
        
        # Create queries table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS queries (
            id INT PRIMARY KEY,
            query TEXT,
            gene_id VARCHAR(255),
            transcript_id VARCHAR(255) UNIQUE,
            FOREIGN KEY (gene_id) REFERENCES genes(Gene)
        )
        """)
        
        # Create isoforms table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS isoforms (
            transcripts VARCHAR(255) PRIMARY KEY,
            gene_id VARCHAR(255),
            FOREIGN KEY (gene_id) REFERENCES genes(Gene)
        )
        """)
        
        # Create results table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS results (
            id INT AUTO_INCREMENT PRIMARY KEY,
            query_id INT,
            pdb_id VARCHAR(255),
            Chain VARCHAR(255),
            e_val VARCHAR(255),
            query_cov FLOAT,
            FOREIGN KEY (query_id) REFERENCES queries(id)
        )
        """)
        
        print("Tables created successfully")
    except Error as err:
        print(f"Error: '{err}'")

def insert_data(connection, queries_data, genes_data, isoforms_data, results_data):
    """Insert data into all tables"""
    cursor = connection.cursor()
    
    try:
        # Insert genes first (referenced by other tables)
        gene_insert_query = """
        INSERT IGNORE INTO genes (Gene, chrom_pos)
        VALUES (%s, %s)
        """
        for gene in genes_data:
            cursor.execute(gene_insert_query, (gene['Gene'], gene['chrom_pos']))
        
        # Insert queries
        query_insert_query = """
        INSERT INTO queries (id, query, gene_id, transcript_id)
        VALUES (%s, %s, %s, %s)
        """
        for query in queries_data:
            cursor.execute(query_insert_query, 
                          (query['id'], query['query'], query['gene_id'], query['transcript_id']))
        
        # Insert isoforms
        isoform_insert_query = """
        INSERT IGNORE INTO isoforms (transcripts, gene_id)
        VALUES (%s, %s)
        """
        for isoform in isoforms_data:
            cursor.execute(isoform_insert_query, (isoform['transcripts'], isoform['gene_id']))
        
        # Insert results
        result_insert_query = """
        INSERT INTO results (query_id, pdb_id, Chain, e_val, query_cov)
        VALUES (%s, %s, %s, %s, %s)
        """
        for result in results_data:
            cursor.execute(result_insert_query, 
                         (result['query_id'], result['pdb_id'], result['Chain'], 
                          result['e_val'], result['query_cov']))
        
        connection.commit()
        print("Data inserted successfully")
    except Error as err:
        print(f"Error: '{err}'")
        connection.rollback()

def main():
    # Database configuration
    host = "localhost"  # or your MySQL server address
    user = "root"
    password = "123456"
    database_name = "blastp"
    
    # Input file path
    input_file = "test_job.out"  # Change to your input file path
    
    # Parse the input file
    queries, genes, isoforms, results = parse_blastp_file(input_file)
    
    # Create database connection
    connection = create_mysql_connection(host, user, password, database_name)
    
    if connection is not None:
        try:
            # Create database (if not exists)
            create_database(connection, database_name)
            
            # Connect to the specific database
            connection.database = database_name
            
            # Create tables
            create_tables(connection)
            
            # Insert data
            insert_data(connection, queries, genes, isoforms, results)
            
            print(f"Database populated successfully with {len(queries)} queries, "
                  f"{len(genes)} genes, {len(isoforms)} isoforms, and {len(results)} results.")
            
        except Error as err:
            print(f"Error: '{err}'")
        finally:
            if connection.is_connected():
                connection.close()
                print("MySQL connection is closed")

if __name__ == '__main__':
    main()

MySQL Database connection successful
Database 'blastp' created successfully
Tables created successfully
Data inserted successfully
Database populated successfully with 771 queries, 305 genes, 771 isoforms, and 172793 results.
MySQL connection is closed
